In [1]:
# importing all the required libraries
from gurobipy import *
import sys
import numpy as np
from collections import defaultdict

In [2]:
# define a function for generating an empty model in gurobi
def get_empty_model():
    m = Model()
    
    return m


In [3]:
# define a function for generating set of variables in gurobi
def get_variables(number_slot_ub, number_patient_ub, number_scenario_ub):
    
    # 1. Service time with nurse: according to the type of patient (High Complexity, Low COmplexity or Same day)
    # 2. Service time with provider
    # 3. Start time with nurse
    # 5. Start time with provider
    # 6. finish time with provider
    # 7. binary variable for patient type/class
    #    A: if High Complexity (HC)
    #    B: if Low Complexity (LC)
    #    C: if same day (SD)
    # 8. binary variable for scheduled slot number, X
    
    # operational varables
    tau_n, tau_p, y_start, y_finish, z_start, z_finish = {}
    for s in range(number_scenario_ub):
        for i in range(number_patient_ub + 1):
            tau_n[(i,s)] = m.addVar(lb = 0,
                                    vtype = GRB.INTEGER, 
                                    name = 'Service time with nurse: Patient = %d, Scenario = %d'%(i, s))
            
            tau_p[(i,s)] = m.addVar(lb = 0,
                                    vtype = GRB.INTEGER, 
                                    name = 'Service time with provider: Patient = %d, Scenario = %d'%(i, s))
            
            y_start[(i,s)] = m.addVar(lb = 0,
                                      vtype = GRB.INTEGER, 
                                      name = 'Start time with nurse: Patient = %d, Scenario = %d'%(i, s))
            
            y_finish[(i,s)] = m.addVar(lb = 0,
                                       vtype = GRB.INTEGER, 
                                       name = 'Finish time with nurse: Patient = %d, Scenario = %d'%(i, s))
            
            z_start[(i,s)] = m.addVar(lb = 0,
                                      vtype = GRB.INTEGER, 
                                      name = 'Start time with provider: Patient = %d, Scenario = %d'%(i, s))
            
            z_finish[(i,s)] = m.addVar(lb = 0,
                                       vtype = GRB.INTEGER, 
                                       name = 'Finish time with provider: Patient = %d, Scenario = %d'%(i, s))
    
    # patient characteristics variables
    A, B, C, X = {}
    for i in range(number_patient_ub):
        A[(i)] = m.addVar(vtype = GRB.BINARY, 
                          name = 'Patient %d is High Complexity'%(i))
        B[(i)] = m.addVar(vtype = GRB.BINARY, 
                          name = 'Patient %d is Low Complexity'%(i))
        C[(i)] = m.addVar(vtype = GRB.BINARY, 
                          name = 'Patient %d is Same-Day scheduled'%(i))
        X[(i)] = m.addVar(lb = 0, ub = number_slot_ub,
                          vtype = GRB.INTEGER, 
                          name = 'Scheduled time slot for Patient %d'%(i))
    
    # adjustments for gurobi
    tau_n        = tupledict(tau_n)
    tau_p        = tupledict(tau_p)
    y_start      = tupledict(y_start)
    y_finish     = tupledict(y_finish)
    z_start      = tupledict(x_start)
    z_finish     = tupledict(x_finish)
    A            = tupledict(A)
    B            = tupledict(B)
    C            = tupledict(C)
    X            = tupledict(X)
    
    # set of variables
    variables = {
        "Serv. time with nurse"    : tau_n,
        "Serv. time with provider" : tau_p,
        "Start time with nurse"    : y_start,
        "Finish time with nurse"   : y_finish,
        "Start time with provider" : z_start,
        "Finish time with provider": z_finish,
        "High-Complexity patient"  : A,
        "Low-Complexity patient"   : B,
        "Same-Day scheduled patient": C,
        "Scheduled time slot for patient": X
    }
    
    # update model and return
    m.update()
    return m, variables
    
    

In [ ]:
# define a function for adding constraints to the gurobi model

def get_contraints(number_slot_ub, number_patient_ub, number_patient, number_scenario_ub, tau_nurse, tau_provider):
    
    # define dict for constraints
    con = {}
    
    # call the model with defined variables and the set of variables too
    m, variables = get_variables(number_slot_ub, number_patient_ub, number_scenario_ub)
    
    # Constraint 1: Reference to start
    variables["Finish time with provider"] = z_finish
    con["1"] = m.addConstrs(z_finish[0, s] == 0 for s in range(number_scenario_ub))
    
    # Constraint 2: Reference to start
    variables["Scheduled time slot for patient"] = X
    con["2"] = m.addConstrs(X[1, s] == 0 for s in range(number_scenario_ub))
    
    # Constraint 3: service time with nurse
    variables["High-Complexity patient"]    = A
    variables["Low-Complexity patient"]     = B
    variables["Same-Day scheduled patient"] = C
    variables["Serv. time with nurse"] = tau_n
    con["3"] = m.addConstrs(tau_n[i,s] == tau_nurse[["HC"][i, s]]*A[i] + tau_nurse[["LC"][i, s]]*B[i] + tau_nurse[["SD"][i, s]]*C[i]
                       for s in range(number_scenario_ub)
                       for i in range(number_patient_ub + 1))
    
    # Constraint 4: service time with provider
    variables["Serv. time with provider"] = tau_p
    con["4"] = m.addConstrs(tau_p[i,s] == tau_provider[["HC"][i, s]]*A[i] + tau_provider[["LC"][i, s]]*B[i] + tau_provider[["SD"][i, s]]*C[i] 
                       for s in range(number_scenario_ub)
                       for i in range(number_patient_ub + 1))
    
    # Constraint 5 and 6: lower bound on start time with nurse (con5 OR con6 whichever is greater)
    variables["Start time with nurse"] = y_start
    variables["Finish time with nurse"] = y_finish
    con["5"] = m.addConstrs(y_start[i,s] >= 15*X[i] 
                        for s in range(number_scenario_ub)
                        for i in range(number_patient_ub + 1))
    con["6"] = m.addConstrs(y_start[i,s] >= y_finish[i-1, s] 
                        for s in range(number_scenario_ub)
                        for i in range(number_patient_ub + 1))
    
    # Constraint 7: finish time with nurse
    con["7"] = m.addConstrs(y_finish == y_start + tau_n)
    
    # Constraint 8 and 9: Lower bound on start time with provider (con8 OR con9 whichever is greater)
    variables["Start time with provider"] = z_start
    variables["Finish time with provider"] = z_finish
    con["5"] = m.addConstrs(z_start[i,s] >= y_finish[i,s] 
                        for s in range(number_scenario_ub)
                        for i in range(number_patient_ub + 1))
    con["6"] = m.addConstrs(y_start[i,s] >= z_finish[i-1, s] 
                        for s in range(number_scenario_ub)
                        for i in range(number_patient_ub + 1))
    
    # Constraint 10: finish time with provider
    con["10"] = m.addConstrs(z_finish == z_start + tau_p)
    
    # Constraint 11, 12, 13, 14: patient number balance and stuff
    con["11"] = m.addConstrs(quicksum(A[i]) == number_patient["HC"])
    con["12"] = m.addConstrs(quicksum(B[i]) == number_patient["LC"])
    con["13"] = m.addConstrs(quicksum(C[i]) == number_patient["SD"])
    con["14"] = m.addConstrs(A[i] + B[i] + C[i] == 1
                            for i in range(number_patient_ub + 1))
    